In [ ]:
import os, requests, glob, re, shutil, time
from google.colab import userdata
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from huggingface_hub import InferenceClient
# NEW CORRECT IMPORT FOR DOCUMENT
from langchain_core.documents import Document
import chromadb # Import chromadb to access Settings


# --- 1. CONFIGURATION ---
MISTRAL_API_KEY = userdata.get('MISTRAL_API_KEY')
HF_TOKEN = userdata.get('HF_TOKEN')
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
DB_PATH = os.path.abspath("./preventis_final_db") # Convert to absolute path


# --- 2. THE RAG PIPELINE (Loading -> Chunking -> Embedding -> Storing) ---
if os.path.exists(DB_PATH):
    shutil.rmtree(DB_PATH)
    time.sleep(1)
os.makedirs(DB_PATH, exist_ok=True) # Ensure the directory is recreated and writable


def setup_rag_ultra():
    pdf_files = glob.glob("/content/sample_data/*.pdf")
    if not pdf_files:
        print("❌ No PDFs found in /content/sample_data/ !")
        return None


    all_docs = []
    for pdf in pdf_files:
        print(f"📖 Loading: {os.path.basename(pdf)}...")
        loader = PyPDFLoader(pdf)
        pages = loader.load()
        for p in pages:
            p.metadata["source"] = os.path.basename(pdf)
            all_docs.append(p)


    # 1. CHUNKING: We cut the text into 1200-char pieces so Article 294 stays whole.
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,
        chunk_overlap=200,
        separators=["\nArticle", "\n\n", "\n", " "]
    )
    chunks = splitter.split_documents(all_docs)
    print(f"✅ Chunking complete: {len(chunks)} blocks created.")


    # 2. EMBEDDING & STORING: MiniLM turns text into math; Chroma stores it.
    embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)


    # Explicitly create a persistent ChromaDB client
    chroma_client = chromadb.PersistentClient(path=DB_PATH)


    return Chroma.from_documents(
        chunks,
        embeddings,
        # Removed persist_directory here to avoid conflict when client is explicitly provided
        client=chroma_client # Pass the explicit ChromaDB client
    )


# --- 3. HYBRID SEARCH LOGIC ---
def get_context(query, vector_db):
    # Keyword Boost (Finds specific article numbers)
    nums = re.findall(r'\d+(?:-\d+)*', query)
    keyword_docs = []
    if nums:
        data = vector_db.get()
        for i, text in enumerate(data['documents']):
            if nums[0] in text:
                keyword_docs.append(Document(page_content=text, metadata=data['metadatas'][i]))


    # Semantic Search
    semantic_docs = vector_db.similarity_search(query, k=4)


    # Merge
    combined = {d.page_content: d for d in (keyword_docs + semantic_docs)}.values()
    return list(combined)


# --- 4. GENERATION ---
def ask_ai(query, vector_db):
    docs = get_context(query, vector_db)
    context_text = "\n".join([f"[{d.metadata['source']}]: {d.page_content}" for d in docs])


    prompt = f"""Role: Expert juridique. Répondez UNIQUEMENT en utilisant le contexte. Citez la source comme [Fichier.pdf]. Langue: Français.


CONTEXT:
{context_text}


QUESTION:
{query}


ANSWER:"""


    # Mistral
    m_res = requests.post("https://api.mistral.ai/v1/chat/completions",
                         headers={"Authorization": f"Bearer {MISTRAL_API_KEY}"},
                         json={"model": "mistral-small-latest", "messages": [{"role": "user", "content": prompt}], "temperature": 0.1}).json()


    # Llama 3.1
    client = InferenceClient(model="meta-llama/Meta-Llama-3.1-8B-Instruct", token=HF_TOKEN)
    l_res = client.chat_completion(messages=[{"role": "user", "content": prompt}], max_tokens=800, temperature=0.1)


    return m_res['choices'][0]['message']['content'], l_res.choices[0].message.content


# --- 5. INTERFACE ---
v_db = setup_rag_ultra()
if v_db:
    while True:
        user_q = input("\n👉 Ask your legal question (or 'exit'): ")
        if user_q.lower() == 'exit': break
        ans_m, ans_l = ask_ai(user_q, v_db)
        print("\n" + "="*50 + f"\n🔵 MISTRAL:\n{ans_m}\n" + "-"*25 + f"\n\U0001f99f LLAMA 3.1:\n{ans_l}\n" + "="*50)

